<a href="https://colab.research.google.com/github/yuu-eguci/flower-stuff-lab/blob/main/yuueguci/mnist_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Sun Jun 27 05:24:50 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P0    32W /  70W |    948MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Google Drive をマウントします。
# NOTE: 左のトコをポチポチやってマウントすることも出来ますが(というかそのほうがラク)
# マウントすることを明示するほうが好みなのでしています。
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Keras のバージョンが結構重要な感あります。確認しておきます。
!pip list | grep Keras

Keras                         2.4.3              
Keras-Preprocessing           1.1.2              


In [ ]:
# NOTE: Colaboratory で module imported but unused とか出す方法あるのかな?
# NOTE: この内容なら、 Colab では pip install 不要です。
from pathlib import Path
import numpy
from PIL import Image
from keras.models import load_model
from keras.datasets import mnist
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import img_to_array, load_img

In [ ]:
class MNISTDataset():
    """MNIST dataset をクラスとして定義しています。
    NOTE: image_shape とか、グローバルスコープで定数で定義するのイヤやん?
    NOTE: インスタンス変数が変わることって無いと思うから、 static class にすべきじゃない?
    """

    def __init__(self):

        # MNIST は 28x28x1(グレースケール)のデータセットです。
        self.image_shape = (28, 28, 1)
        # クラス数は当然 0~9 の10個です。
        self.num_classes = 10

    def get_batch(self):

        # NOTE: このメソッド名はサンプルママです。
        #       強化学習の方式をオンライン学習といい、既存データセットから学習することをバッチ学習という。
        #       「バッチ学習」用のデータだから batch なのかな。

        (x_train, y_train), (x_test, y_test) = mnist.load_data()

        # preprocess により 0~1 の値に変換します。
        # label_data(正解データ)は one-hot ベクトルへ変換します。
        # NOTE: one-hot ベクトルは、ラベルの数(10)と等しい数のベクトルです。
        # NOTE: ここはよくわかっていない。
        x_train, x_test = [self.preprocess(d) for d in [x_train, x_test]]
        y_train, y_test = [self.preprocess(d, label_data=True) for d in
                           [y_train, y_test]]

        return x_train, y_train, x_test, y_test

    def preprocess(self, data, label_data=False):

        if label_data:
            # convert class vectors to binary class matrices
            data = to_categorical(data, self.num_classes)
        else:
            data = data.astype('float32')
            data /= 255  # convert the value to 0~1 scale
            shape = (data.shape[0],) + self.image_shape  # add dataset length
            data = data.reshape(shape)

        return data

In [ ]:
# hdf5 を読み込み、 mnist_fitting で作った model が読み込めていることを確認します。
model = load_model('/content/drive/MyDrive/hdf5/mnist_fitting.hdf5')

# MNIST を読み込みます。
# NOTE: evaluate するためにデータセットが必要です。
mnist_dataset = MNISTDataset()

# MNIST からデータセットを取得します。
x_train, y_train, x_test, y_test = mnist_dataset.get_batch()

# 結果を閲覧します。
score = model.evaluate(
    x_test,
    y_test,
    verbose=0,
)
print(f'Test loss: {score[0]}')
print(f'Test accuracy: {score[1]}')

Test loss: 0.02797297015786171
Test accuracy: 0.9916999936103821


In [ ]:
# FIXME: hdf5 が正常にロードできることは確認しました。
#        が、 MNIST の prediction を成功することができません。
#        ValueError: Input 0 of layer sequential_5 is incompatible with the layer:
#        expected axis -1 of input shape to have value 1 but received input with shape (None, 28, 28, 3)
model.predict(xxx)

ValueError: ignored